# This code is to generate G-Code for the arm designed

In [67]:
import sympy as sym
import numpy as np
import csv
import pandas as pd
pi=np.pi
import time
start_time = time.time()

## Lengths

In [68]:
L1=3.68#cm Base Height
L2=36#cm
L3=19.8#cm
L4=14#cm
L5=13.3#cm

## Joint Movement Freedom

## Joint 1

In [69]:
D1min=0
D1max=360

## Joint 2

In [70]:
D2min=0
D2max=230

## Joint 3

In [71]:
D3min=0
D3max=210

## Joint 4

In [72]:
D4min=0
D4max=180

## Joint 5

In [73]:
D5min=0
D5max=360

## Define Angles

In [74]:
q1=np.linspace(D1min,D1max,4)
q2=np.linspace(D2min,D2max,4)
q3=np.linspace(D3min,D3max,4)
q4=np.linspace(D4min,D4max,4)
q5=np.linspace(D5min,D5max,4)
q6=0

q1r=np.deg2rad(q1)
q2r=np.deg2rad(q2)
q3r=np.deg2rad(q3)
q4r=np.deg2rad(q4)
q5r=np.deg2rad(q5)

In [75]:
# Create Modified DH parameters
alpha0=0
alpha1=-pi/2
alpha2=0
alpha3=0
alpha4=-pi/2
alpha5=0

a0=0
a1=0
a2=L2
a3=0
a4=0
a5=0

d1=L1
d2=0
d3=0
d4=L3
d5=0
d6=L4+L5

## Transfer Matrix

In [76]:
# Define Modified DH Transformation matrix
def TF_Matrix(alpha,a,d,q):
    TF=([[np.cos(q),-np.sin(q),0,a],
    [np.sin(q)*np.cos(alpha),np.cos(q)*np.cos(alpha),-np.sin(alpha),-np.sin(alpha)*d],
    [np.sin(q)*np.sin(alpha),np.cos(q)*np.sin(alpha),np.cos(alpha),np.cos(alpha)*d],
    [0,0,0,1]])
    return TF

In [77]:
#Constant Transform
T5_EE=np.array(TF_Matrix(alpha5,a5,d6,q6))
P = [52.8,0.0,63.8,1]

In [78]:
# Create individual transformation matrices
count=0
with open('transforms.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['q1','q2','q3','q4','q5','x','y','z','1'])
    for i in q1r:
        T0_1=np.array(TF_Matrix(alpha0,a0,d1,i))
        for k in q2r:
            T1_2=np.array(TF_Matrix(alpha1,a1,d2,k))
            for j in q3r:
                T2_3=np.array(TF_Matrix(alpha2,a2,d3,j))
                for l in q4r:
                    T3_4=np.array(TF_Matrix(alpha3,a3,d4,l))
                    for h in q5r:
                        T4_5=np.array(TF_Matrix(alpha4,a4,d5,h))
                        T0_EE=T0_1*T1_2*T2_3*T3_4*T4_5*T5_EE
                        P_new = sym.simplify(T0_EE * P)
                        data=(i,k,j,l,h,round(P_new[0],3),round(P_new[1],3),round(P_new[2],3),round(P_new[3],3))
                        writer.writerow(data)

In [79]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.946122407913208 seconds ---
